In [1]:
#Library Imports
import numpy as np
import pandas as pd

In [7]:
data = pd.read_csv('data/ml-100k/u.data',sep='\t',names=['user_id','movie_id','rating','timestamp'])
#movies = pd.read_csv('data/ml-latest/movies.csv')

In [9]:
print(data.count())

user_id      100000
movie_id     100000
rating       100000
timestamp    100000
dtype: int64


In [10]:
data.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [11]:
#movies.head()

In [12]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data,test_size=0.25, random_state=42)
#movies_train, movies_test = train_test_split(movies,test_size=0.25, random_state=42)

In [31]:
totalUsers = data.user_id.unique().shape[0]
totalMovies = data.movie_id.unique().shape[0]

print(totalUsers,totalMovies)

943 1682


In [48]:
# Create User-Item Matrix
train_matrix = np.zeros((totalUsers,totalMovies))
print(train_matrix.shape)
for line in data_train.itertuples():
    train_matrix[line[1]-1,line[2]-1] = line[3]

test_matrix = np.zeros((totalUsers,totalMovies))
print(test_matrix.shape)
for line in data_test.itertuples():
    test_matrix[line[1]-1,line[2]-1] = line[3]

(943, 1682)
(943, 1682)


In [33]:
# Calculate Cosine Similarity
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_matrix, metric='cosine')
item_similarity = pairwise_distances(train_matrix.T, metric='cosine')

In [45]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [46]:
item_prediction = predict(train_matrix, item_similarity, type='item')
user_prediction = predict(train_matrix, user_similarity, type='user')

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [51]:
print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))

User-based CF RMSE: 3.125042691772117
Item-based CF RMSE: 3.453295523246612
